# Convex expectile regression

   + Author : Sheng Dai (sheng.dai@aalto.fi)
   + Date : June 12, 2020

References:

[1] Kuosmanen, T., Zhou, X. and Dai, S. (2020). How much climate policy has cost for OECD countries?, World Development, 125, p. 104681.

[2] Dai, S., Zhou, X. and Kuosmanen, T. (2020). Forward-looking assessment of the GHG abatement cost: Application to China, Energy Economics, 88, p. 104758.

In [ ]:
from pystoned import CQER
import pandas as pd
import numpy as np

In [ ]:
# import data
url = 'https://raw.githubusercontent.com/ds2010/pyStoNED-Tutorials/master/Data/data.csv'
df = pd.read_csv(url, error_bad_lines=False)

In [ ]:
# output
y  = df['TOTEX']

# inputs
x1  = df['Energy']
x1  = np.asmatrix(x1).T
x2  = df['Length']
x2  = np.asmatrix(x2).T
x3  = df['Customers']
x3  = np.asmatrix(x3).T
x   = np.concatenate((x1, x2, x3), axis=1)

In [ ]:
# calculate the quantile model
cet   = "mult"
fun   = "cost"
rts   = "crs"
tau   = 0.9

model = CQER.cqr(y, x, tau, cet, fun, rts)

from pyomo.environ import SolverManagerFactory
solver_manager = SolverManagerFactory('neos')
results = solver_manager.solve(model, opt='knitro', tee=True)

In [ ]:
# display estiamtes
model.em.display()
model.ep.display() 
model.b.display() 

In [ ]:
# retrive the residuals (eminos)
val = list(model.em[:].value)
eminos = np.asarray(val)
eminos

In [ ]:
# retrive the residuals (eplus)
val = list(model.ep[:].value)
eplus = np.asarray(val)
eplus

In [ ]:
# retrive the beta
ind = list(model.b)
val = list(model.b[:,:].value)
beta= np.asarray([ i + tuple([j]) for i, j in zip(ind, val)]) 

import pandas as pd
beta = pd.DataFrame(beta, columns = ['Name', 'Key', 'Value'])
beta = beta.pivot(index='Name', columns='Key', values='Value')
beta.columns = ['b1', 'b2', 'b3']

beta